In [1]:
import subprocess
import shlex
import json
import time 
import os
import sys

class Bitrate :
    def __init__(self, input_data, output, timeout=10):
        self.input_data = input_data
        self.output = output
        self.timeout = timeout
    
    def get_stream(self):
        timeout = time.time() + self.timeout
        cmd = "ffmpeg -i "+self.input_data+" -hide_banner -y -c copy "+self.output
        process = subprocess.Popen(cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True, shell=True)
        for line in process.stdout:
            print(line)
            if time.time() > timeout:
                break
        process.communicate(input='q')
        
    def manual_calculation(self):
        data = subprocess.run(shlex.split(f'ffprobe -select_streams v:0 -show_entries packet=size,duration -of compact=p=0:nk=1 -print_format json '+self.output), stdout=subprocess.PIPE).stdout
        dict = json.loads(data)
        sum_packets_size = 0
        sum_packets_duration = 0
        for p in dict['packets']:
            sum_packets_size += float(p['size'])
            sum_packets_duration += float(p['duration'])
        bit_rate = round((sum_packets_size / sum_packets_duration) * 8 * 1000)
        print(f'bit_rate = {bit_rate}')        

In [2]:
# Streaming rtsp
br = Bitrate(
    input_data="rtsp://wowzaec2demo.streamlock.net/vod/mp4:BigBuckBunny_115k.mov",
    output="output.mp4",
    timeout=30
)

In [3]:
br.get_stream()

[rtsp @ 000001a50e40ecc0] max delay reached. need to consume packet

[rtsp @ 000001a50e40ecc0] RTP: missed 1 packets

[rtsp @ 000001a50e40ecc0] max delay reached. need to consume packet

[rtsp @ 000001a50e40ecc0] RTP: missed 3 packets

Input #0, rtsp, from 'rtsp://wowzaec2demo.streamlock.net/vod/mp4:BigBuckBunny_115k.mov':

  Metadata:

    title           : BigBuckBunny_115k.mov

  Duration: 00:10:34.63, start: 0.000000, bitrate: N/A

  Stream #0:0: Audio: aac (LC), 12000 Hz, stereo, fltp

  Stream #0:1: Video: h264 (High), yuv420p(progressive), 240x160 [SAR 32:27 DAR 16:9], 24 fps, 24 tbr, 90k tbn

Output #0, mp4, to 'output.mp4':

  Metadata:

    title           : BigBuckBunny_115k.mov

    encoder         : Lavf59.5.100

  Stream #0:0: Video: h264 (High) (avc1 / 0x31637661), yuv420p(progressive), 240x160 [SAR 32:27 DAR 16:9], q=2-31, 24 fps, 24 tbr, 90k tbn

  Stream #0:1: Audio: aac (LC) (mp4a / 0x6134706D), 12000 Hz, stereo, fltp

Stream mapping:

  Stream #0:1 -> #0:0 (copy)

 

frame=  399 fps= 24 q=-1.0 size=       0kB time=00:00:17.66 bitrate=   0.0kbits/s speed=1.07x    

[rtsp @ 000001a50e40ecc0] max delay reached. need to consume packet

[rtsp @ 000001a50e40ecc0] RTP: missed 2 packets

[rtsp @ 000001a50e40ecc0] max delay reached. need to consume packet

[rtsp @ 000001a50e40ecc0] RTP: missed 5 packets

frame=  413 fps= 24 q=-1.0 size=       0kB time=00:00:18.34 bitrate=   0.0kbits/s speed=1.08x    

frame=  427 fps= 24 q=-1.0 size=       0kB time=00:00:18.79 bitrate=   0.0kbits/s speed=1.07x    

[rtsp @ 000001a50e40ecc0] max delay reached. need to consume packet

[rtsp @ 000001a50e40ecc0] RTP: missed 2 packets

[rtsp @ 000001a50e40ecc0] max delay reached. need to consume packet

[rtsp @ 000001a50e40ecc0] RTP: missed 2 packets

[rtsp @ 000001a50e40ecc0] max delay reached. need to consume packet

[rtsp @ 000001a50e40ecc0] RTP: missed 1 packets

frame=  438 fps= 24 q=-1.0 size=       0kB time=00:00:19.29 bitrate=   0.0kbits/s speed=1.06x    

frame=  450 fp

In [4]:
br.manual_calculation()

bit_rate = 580


In [5]:
# Streaming http
br = Bitrate(
    input_data="http://commondatastorage.googleapis.com/gtv-videos-bucket/sample/BigBuckBunny.mp4",
    output="output1.mp4",
    timeout=30
)

In [6]:
br.get_stream()

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'http://commondatastorage.googleapis.com/gtv-videos-bucket/sample/BigBuckBunny.mp4':

  Metadata:

    major_brand     : mp42

    minor_version   : 0

    compatible_brands: isomavc1mp42

    creation_time   : 2010-01-10T08:29:06.000000Z

  Duration: 00:09:56.47, start: 0.000000, bitrate: 2119 kb/s

  Stream #0:0[0x1](und): Audio: aac (LC) (mp4a / 0x6134706D), 44100 Hz, stereo, fltp, 125 kb/s (default)

    Metadata:

      creation_time   : 2010-01-10T08:29:06.000000Z

      handler_name    : (C) 2007 Google Inc. v08.13.2007.

      vendor_id       : [0][0][0][0]

  Stream #0:1[0x2](und): Video: h264 (High) (avc1 / 0x31637661), yuv420p, 1280x720 [SAR 1:1 DAR 16:9], 1991 kb/s, 24 fps, 24 tbr, 24k tbn (default)

    Metadata:

      creation_time   : 2010-01-10T08:29:06.000000Z

      handler_name    : (C) 2007 Google Inc. v08.13.2007.

      vendor_id       : [0][0][0][0]

Output #0, mp4, to 'output1.mp4':

  Metadata:

    major_brand     : mp4

In [7]:
br.manual_calculation()

bit_rate = 102430


In [8]:
# Offline data
br = Bitrate(
    input_data="mini-data.mp4",
    output="output2.mp4",
    timeout=30
)

In [9]:
br.get_stream()

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'mini-data.mp4':

  Metadata:

    major_brand     : isom

    minor_version   : 512

    compatible_brands: isomiso2avc1mp41

    encoder         : Lavf55.48.100

    copyright       : copyright

    description     : description

  Duration: 00:00:09.71, start: 0.000000, bitrate: 251 kb/s

  Stream #0:0[0x1](und): Video: h264 (Constrained Baseline) (avc1 / 0x31637661), yuv420p(tv, bt709), 480x270, 180 kb/s, 30 fps, 30 tbr, 15360 tbn (default)

    Metadata:

      handler_name    : VideoHandler

      vendor_id       : [0][0][0][0]

  Stream #0:1[0x2](und): Audio: aac (LC) (mp4a / 0x6134706D), 44100 Hz, mono, fltp, 64 kb/s (default)

    Metadata:

      handler_name    : SoundHandler

      vendor_id       : [0][0][0][0]

Output #0, mp4, to 'output2.mp4':

  Metadata:

    major_brand     : isom

    minor_version   : 512

    compatible_brands: isomiso2avc1mp41

    description     : description

    copyright       : copyright

    encoder  

In [10]:
br.manual_calculation()

bit_rate = 11721
